In [1]:
# Import packages
import os
import numpy as np 
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist, pdist, squareform

In [2]:
def load_data(path, name):
    '''
    --------------------
    Prepare data
    --------------------
    Parameters: 
    weights: Current set of weights
    biases: Current set of biases
    gradients: Current set of gradients
    learning_rate: parameter to guide SGD step size
    --------------------
    Output: 
    Updated weights and biases
    --------------------
    '''
    data = np.loadtxt(os.path.join(path, name))
    X, Y = data[:, 1:], data[:, 0]

    return(X, Y)

In [3]:
def shuffle_data(X, Y):
    '''
    --------------------
    Prepare data
    --------------------
    Parameters:
    weights: Current set of weights
    biases: Current set of biases
    gradients: Current set of gradients
    learning_rate: parameter to guide SGD step size
    --------------------
    Output:
    Updated weights and biases
    --------------------
    '''
    # Data is currently unshuffled; we should shuffle
    # each X[i] with its corresponding y[i]
    perm = np.random.permutation(max(Y.shape))
    X = X[perm, :]
    Y = Y[perm]

    return(X, Y)

In [4]:
def split_data(X, Y, train_percent):
    '''
    --------------------
    Prepare data
    --------------------
    Parameters: 
    weights: Current set of weights
    biases: Current set of biases
    gradients: Current set of gradients
    learning_rate: parameter to guide SGD step size
    --------------------
    Output: 
    Updated weights and biases
    --------------------
    '''
    # Calculate no. of training examples based on user specified percentage
    # Here we use 2/3, 1/3 by default as required by the assignment
    n_train = round(train_percent*max(Y.shape))
    
    # Filter the dataframe to get training and testing rows
    X_train = X[:n_train, :]
    Y_train = Y[:n_train]
    
    # Validation set
    X_val = X[n_train:, :]
    Y_val = Y[n_train:]
    
    # Return statement
    return(X_train, X_val, Y_train, Y_val)

In [5]:
def get_polynomial_kernel(X, X_, d):
    '''
    --------------------
    Prepare data
    --------------------
    Parameters: 
    weights: Current set of weights
    biases: Current set of biases
    gradients: Current set of gradients
    learning_rate: parameter to guide SGD step size
    --------------------
    Output: 
    Updated weights and biases
    --------------------
    '''
    return(np.power(np.dot(X, X.T), d))

In [6]:
def get_gaussian_kernel(X, X_, c):
    '''
    --------------------
    Prepare data
    --------------------
    Parameters: 
    weights: Current set of weights
    biases: Current set of biases
    gradients: Current set of gradients
    learning_rate: parameter to guide SGD step size
    --------------------
    Output: 
    Updated weights and biases
    --------------------
    '''
    # Compute pairwise distances
    K = np.einsum('ij,ij->i',X, X)[:,None] + np.einsum('ij,ij->i',X_,X_) - 2*np.dot(X,X_.T)
    
    # Then apply parameter c
    K = np.exp(K*c)
    
    # Return statement
    return(K)

In [7]:
def get_accuracy(target, pred):
    '''
    --------------------
    Prepare data
    --------------------
    Parameters: 
    weights: Current set of weights
    biases: Current set of biases
    gradients: Current set of gradients
    learning_rate: parameter to guide SGD step size
    --------------------
    Output: 
    Updated weights and biases
    --------------------
    '''
    return np.sum(target==pred)/max(target.shape)

In [8]:
def get_results(history):
    '''
    --------------------
    Get results
    --------------------
    Parameters: 
    weights: Current set of weights
    biases: Current set of biases
    gradients: Current set of gradients
    learning_rate: parameter to guide SGD step size
    --------------------
    Output: 
    Updated weights and biases
    --------------------
    '''
    # Store results
    best_epoch = np.array(history["dev_accuracies"]).argmax()
    best_training_accuracy = history['accuracies'][best_epoch]
    best_dev_accuracy = history['dev_accuracies'][best_epoch]
    
    # Display results
    print(f"best training accuracy: {history['accuracies'][best_epoch]}")
    print(f"best dev accuracy: {history['dev_accuracies'][best_epoch]}")
    print(f"best epoch: {best_epoch}")
    
    return(best_epoch, best_training_accuracy, best_dev_accuracy)

In [9]:
def train_kernel_perceptron(X_train, Y_train, X_dev, Y_dev, epochs, lr, kernel, kernel_args, n_classes):
    '''
    --------------------
    Kernel perceptron algorithm
    --------------------
    Parameters: 
    X: Numpy array of training features (shape = 784 X n)
    y: Binary (1/-1) training label (shape = n X 1)
    --------------------
    Output: 
    w: trained weights
    b: trained biases
    y_preds: predictions 
    --------------------
    '''
    # Store a record of training and validation accuracies at each epoch
    history = {
        "train_accuracies": [],
        "val_accuracies": []
    }
    
    # Transform X according to the user specified kernel
    if kernel == 'polynomial':
        X_train = get_polynomial_kernel(X_train, X_train, **kernel_args)
    elif kernel == 'gaussian':
        X_train = get_gaussian_kernel(X_train, X_train, **kernel_args)
    
    # Initialize alpha weights
    A = np.zeros((n_classes, X_train.shape[0]))
    
    # Initialize the best accuracy to 0 and update this during training
    best_accuracy = 0
    
    # Store the number of samples for accuracy calculations
    n_samples = max(Y_train.shape)
    
    # Run for a fixed number of epochs
    for epoch in range(epochs):
        
        # Print shapes as a test
        print('This is epoch number: {}'.format(epoch))
        
        # Track number of mistakes for this epoch here
        mistakes = 0

        # Do this for each example in the dataset
        for i in range(X_train.shape[0]):

            # Compute the prediction with the current weights:
            # dim(A.T) --> (10, 6199), dim(X_train[i, :]) ---> (6199, 1) ====> dim(y_hat) --> 10 X 1
            Y_hat = A @ X_train[i, :]
            
            # Compute predictions check
            Y = np.full(n_classes, -1)
            Y[int(Y_train[i]) - 1] = 1
            
            # Compute sign
            signs = np.ones(Y_hat.shape)
            signs[Y_hat <= 0] = -1
            
            # Check if the prediction is correct against the labels
            # If it is correct we don't need to make any updates: we just move to the next iteration
            # If it is not correct then we update the weights and biases in the direction of the label
            A[Y*Y_hat <= 0, i] -= (signs[Y*Y_hat <= 0]) 
            
            # Increment mistakes counter
            mistakes += int(np.argmax(Y_hat) + 1 != Y_train[i])
        
        # Print the number of mistakes at the end of each epoch...
        print('The number of mistakes made on epoch {} is {} from {} examples...'.format(epoch, mistakes, n_samples))
        
    # Return statement
    return(history)

In [10]:
def run_kernel_perceptron_training(epochs, lr, 
                                   data_path = 'data', 
                                   name = 'zipcombo.dat', 
                                   kernel = 'polynomial', 
                                   d = 5, n_classes=10):
    '''
    --------------------
    Run perceptron algorithm to get a base-line
    --------------------
    Parameters: 
    X: Numpy array of training features (shape = 784 X n)
    y: Binary (1/0) training label (shape = n X 1)
    --------------------
    Output: 
    w: trained weights
    y_preds: predictions
    --------------------
    '''
    # Set the random seed for random number generator to ensure reproducibility
    np.random.seed(132089)

    # Prepare data for the perceptron
    X, Y = load_data(data_path, name)
    
    # Shuffle the dataset before splitting it
    X, Y = shuffle_data(X, Y)
    
    # Split the data into training and validation set 
    X_train, X_val, Y_train, Y_val = split_data(X, Y, 0.66666)
    
    # Construct kernel arguments dictionary
    if kernel == 'polynomial': kernel_args = {'d': d}

    # Call the perceptron training with the given epochs
    train_kernel_perceptron(X_train, Y_train, X_val, Y_val, epochs, lr, kernel, kernel_args, n_classes)
    
    # Get results from history
    # best_epoch, best_training_accuracy, best_dev_accuracy = get_results(history)
    
    # Return statement
    # return(best_epoch, best_training_accuracy, best_dev_accuracy, history)

In [11]:
def main(epochs = 500, lr = 1):
    '''
    --------------------
    Main training loop
    --------------------
    Parameters: 
    weights: Current set of weights
    biases: Current set of biases
    gradients: Current set of gradients
    learning_rate: parameter to guide SGD step size
    --------------------
    Output: 
    Updated weights and biases
    --------------------
    '''
    # Call training function
    run_kernel_perceptron_training(epochs, lr)

In [12]:
main()

This is epoch number: 0
The number of mistakes made on epoch 0 is 1496 from 6199 examples...
This is epoch number: 1
The number of mistakes made on epoch 1 is 1110 from 6199 examples...
This is epoch number: 2
The number of mistakes made on epoch 2 is 1075 from 6199 examples...
This is epoch number: 3
The number of mistakes made on epoch 3 is 1063 from 6199 examples...
This is epoch number: 4
The number of mistakes made on epoch 4 is 1057 from 6199 examples...
This is epoch number: 5
The number of mistakes made on epoch 5 is 1056 from 6199 examples...
This is epoch number: 6
The number of mistakes made on epoch 6 is 1057 from 6199 examples...
This is epoch number: 7
The number of mistakes made on epoch 7 is 1056 from 6199 examples...
This is epoch number: 8
The number of mistakes made on epoch 8 is 1055 from 6199 examples...
This is epoch number: 9
The number of mistakes made on epoch 9 is 1055 from 6199 examples...
This is epoch number: 10
The number of mistakes made on epoch 10 is 10

The number of mistakes made on epoch 87 is 1053 from 6199 examples...
This is epoch number: 88
The number of mistakes made on epoch 88 is 1053 from 6199 examples...
This is epoch number: 89
The number of mistakes made on epoch 89 is 1053 from 6199 examples...
This is epoch number: 90
The number of mistakes made on epoch 90 is 1053 from 6199 examples...
This is epoch number: 91
The number of mistakes made on epoch 91 is 1053 from 6199 examples...
This is epoch number: 92
The number of mistakes made on epoch 92 is 1053 from 6199 examples...
This is epoch number: 93
The number of mistakes made on epoch 93 is 1053 from 6199 examples...
This is epoch number: 94
The number of mistakes made on epoch 94 is 1053 from 6199 examples...
This is epoch number: 95
The number of mistakes made on epoch 95 is 1053 from 6199 examples...
This is epoch number: 96
The number of mistakes made on epoch 96 is 1053 from 6199 examples...
This is epoch number: 97
The number of mistakes made on epoch 97 is 1053 fr

KeyboardInterrupt: 

In [ ]:
# Make plots of losses to see whether it is converging
# Shapes of all matrices
# Take a look at the actual data in each matrix
# Take a look at each weight
# Take a look at whether kernel arguments are being sent in correctly
# Time run
# Make notation more explicit about kernel